In [ ]:
# import geopandas as gpd
# import pandas as pd
# import json
# import requests
# import time
# from tqdm import tqdm

# # load county shapefiles

# shapefile_path = '../data/raw/tl_2024_us_county/tl_2024_us_county.shp'
# counties = gpd.read_file(shapefile_path)
# counties = counties.to_crs(epsg=4326)

# # load disaster declarations
# df = pd.read_csv("../data/raw/DisasterDeclarationsSummaries.csv")

# df["GEOID"] = df["fipsStateCode"].astype(str).str.zfill(2) + df["fipsCountyCode"].astype(str).str.zfill(3)

# GEOIDS = "../data/cleaned/county_disaster_counts_by_year.json"

# NUM2ABR = "../data/raw/num2abr.txt"


# with open(GEOIDS, 'r') as f:
#     data = json.load(f)

# statedict = {}

# with open(NUM2ABR, 'r') as s:
#     for line in s:
#         parts = line.strip().split(maxsplit=1)
#         if len(parts) == 2:
#             code, state = parts
#             statedict[code] = state

# # print(statedict.keys())
# # print(df['fipsStateCode'])
# df['fipsStateCode'] = df['fipsStateCode'].astype(str).map(statedict)
# df = df.dropna()
# # print(df['fipsStateCode'])

# df['fipsStateCode'] = (df['fipsStateCode'] + '-')

# df['URLS'] = (df['fipsStateCode'].astype(str) + df['fipsCountyCode'].astype(str).str.zfill(3))

# # print(df['URLS'])

# # test_url = "https://www.ncei.noaa.gov/access/monitoring/climate-at-a-glance/county/time-series/TX-059/tavg/1/3/1895-2025/zingchart-config.js"
# # r = requests.get(test_url)
# # print(r.text)

# records = []
# base_url = "https://www.ncei.noaa.gov/access/monitoring/climate-at-a-glance/county/time-series/{code}/tavg/1/3/1895-2025.json"

# r = requests.get("https://www.ncei.noaa.gov/access/monitoring/climate-at-a-glance/county/time-series/WV-043/tavg/1/3/1895-2025.json")
# data = r.json()
# print(data)

# for code in tqdm(df['URLS']):
#     temp_url = base_url.replace("{code}", code)
#     r = requests.get(temp_url)

#     if r.status_code != 200:
#         print(f"L response --> {code}: {r.status_code}")
#         continue  # Skip this one if response isn't OK

#     try:
#         data = r.json()
#     except json.JSONDecodeError:
#         print(f"fucked up json --> {code}")
#         continue

#     county_entry = {
#         "County Code": code,
#         "County Name": data["description"]["title"].split(',', 1)[0],
#         "data": {}
#     }
#     for year_month, entry in data["data"].items():
#         year = year_month[:4]
#         value = entry.get("value")
#         if value is not None:
#             county_entry["data"][year] = value

#     records.append(county_entry)


# with open("county_temperature_data.json", "w") as f:
#     json.dump(records, f, indent=2)

# print('done fr')
    



C:\Users\huds0\AppData\Local\Temp\ipykernel_10736\706002477.py:15: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/raw/DisasterDeclarationsSummaries.csv")


{'description': {'title': 'Lincoln County, West Virginia March Average Temperature', 'missing': -99, 'units': 'Degrees Fahrenheit'}, 'data': {'189503': {'value': 42.8}, '189603': {'value': 38.5}, '189703': {'value': 47.6}, '189803': {'value': 48.8}, '189903': {'value': 44.5}, '190003': {'value': 42.4}, '190103': {'value': 45.8}, '190203': {'value': 46.2}, '190303': {'value': 53.8}, '190403': {'value': 47}, '190503': {'value': 51.7}, '190603': {'value': 40}, '190703': {'value': 53.4}, '190803': {'value': 51.4}, '190903': {'value': 43.6}, '191003': {'value': 52.6}, '191103': {'value': 45.6}, '191203': {'value': 42.3}, '191303': {'value': 46.7}, '191403': {'value': 40.9}, '191503': {'value': 37.1}, '191603': {'value': 42.7}, '191703': {'value': 45.2}, '191803': {'value': 49.9}, '191903': {'value': 47.1}, '192003': {'value': 43.7}, '192103': {'value': 55.4}, '192203': {'value': 47.3}, '192303': {'value': 45.4}, '192403': {'value': 41.4}, '192503': {'value': 47.1}, '192603': {'value': 37.4}

  8%|▊         | 229/2766 [01:54<13:57,  3.03it/s]

L response --> MN-000: 404


  8%|▊         | 230/2766 [01:54<13:55,  3.04it/s]

...

L response --> ND-000: 404


 99%|█████████▊| 2731/2766 [22:13<00:10,  3.24it/s]

L response --> ND-000: 404


100%|██████████| 2766/2766 [22:26<00:00,  2.05it/s]


done fr
